### 可视化

In [ ]:
from torchvision import transforms
from model.Normalization import *

transform=transforms.Compose([
    # transforms.Normalize((0,),(12.3,))
])
# transform=None

In [ ]:
from model.RFID_Dataset import RFID_Dataset

# 训练集
train_dataset=RFID_Dataset(
    r"data\RFID\dataset\all",
    # r"data\RFID\dataset\train",
    T=32,
    step=1,
    transform=transform,
)
print(f"训练集的数据个数: {len(train_dataset)}")

In [ ]:
from model.RFID_Dataset import RFID_Dataset

# 测试集
test_dataset=RFID_Dataset(
    # r"data\RFID\dataset\eval",
    r"./output/improve_v1",
    T=32,
    step=1,
    # transform=transform,
)
print(f"测试集的数据个数: {len(test_dataset)}")

In [ ]:
from utils.FeatureUtils import *

train_input,_=train_dataset[:]
test_input,_=test_dataset[:]

# 降维

pca = get_pca_model(train_input, target=2)
train_output = use_pca_model(pca, train_input)
test_output = use_pca_model(pca, test_input)

In [ ]:
# 单独的PCA降维

train_output=pca_reduce(train_input)
test_output=pca_reduce(test_input)

In [ ]:
# 单独TSNE降维

train_output=tsne_reduce(train_input)
test_output=tsne_reduce(test_input)

In [ ]:
# 绘制图像

from utils.DataUtils.Visualization import plot_classes_scatter

plot_classes_scatter(train_output, train_dataset.labels,title="Train Data")
plot_classes_scatter(test_output, test_dataset.labels,title="Test Data")

In [ ]:
# 数据分布对比

from utils.DataUtils.Visualization import plot_classes_scatter
import numpy as np

datas=np.concatenate([train_output,test_output],axis=0)
temp_labels=(test_dataset.labels)+3
labels=np.concatenate([train_dataset.labels,temp_labels],axis=0)
plot_classes_scatter(datas,labels)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde


def tensor_to_numpy(tensor):
    """统一处理不同框架的张量转NumPy"""
    if hasattr(tensor, "asnumpy"):  # MindSpore张量
        return tensor.asnumpy()
    elif hasattr(tensor, "numpy"):  # PyTorch张量
        return tensor.detach().cpu().numpy()
    return np.array(tensor)


def plot_tensor_density(
    data,
    labels=None,
    colors=None,
    title="Tensor Density Plot",
    xlabel="Value",
    ylabel="Density",
    bandwidth=0.5,
    sample_axis=None,
    max_slices=12,  # 新增：最大允许绘制的切片数
    verbose=True,  # 是否显示警告信息
    save_path=None,
):
    """
    支持高阶张量的密度图绘制（带维度长度限制）

    新增参数:
    max_slices (int): 允许绘制的最大切片数（超过时自动抽样或跳过）
    verbose (bool): 是否提示维度长度警告
    """
    plt.figure(figsize=(10, 6))
    datasets = []
    auto_labels = []

    # 处理输入数据
    if not isinstance(data, (list, tuple)):
        data = [data]

    # 步骤1：展平数据并处理sample_axis
    for tensor in data:
        arr = tensor_to_numpy(tensor)
        if sample_axis is not None:
            # 获取目标维度长度
            dim_length = arr.shape[sample_axis]

            # 跳过超长维度
            if dim_length > max_slices:
                if verbose:
                    print(
                        f"Warning: Dimension length {dim_length} exceeds max_slices={max_slices}. "
                        f"Skipping this tensor's sample_axis={sample_axis}"
                    )
                continue  # 跳过整个张量的处理

            # 正常处理可接受维度
            arr = np.moveaxis(arr, sample_axis, 0)  # 将目标轴移到第一维
            slices = [x_slice.reshape(-1) for x_slice in arr]  # 每个切片展平为1D
            datasets.extend(slices)
            auto_labels.extend([f"Slice {i}" for i in range(dim_length)])
        else:
            datasets.append(arr.reshape(-1))

    # 步骤2：处理标签和颜色
    num_datasets = len(datasets)
    if labels is None:
        labels = auto_labels
    elif len(labels) < num_datasets:
        labels += [f"Slice {i}" for i in range(len(labels), num_datasets)]

    if colors is None:
        colors = plt.cm.tab10(range(num_datasets))
    elif len(colors) < num_datasets:
        colors = list(colors) + [
            colors[i % len(colors)] for i in range(len(colors), num_datasets)
        ]

    # 步骤3：绘制每个切片的KDE
    for i, (d, label, color) in enumerate(zip(datasets, labels, colors)):
        try:
            kde = gaussian_kde(d)
            kde.set_bandwidth(bandwidth)
            x_range = np.linspace(d.min(), d.max(), 1000)
            plt.plot(
                x_range, kde(x_range), label=label, color=color, linewidth=2, alpha=0.7
            )
        except np.linalg.LinAlgError:
            print(f"Skipping {label} due to singular matrix error (insufficient data)")

    # 调整布局
    plt.title(title, fontsize=14)
    plt.xlabel(xlabel, fontsize=12)
    plt.ylabel(ylabel, fontsize=12)
    plt.grid(alpha=0.3)
    plt.legend(fontsize=10, bbox_to_anchor=(1.05, 1), loc="upper left")

    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
    plt.show()

In [ ]:

datas=train_dataset[:][0]
plot_tensor_density(
    datas,
    sample_axis=3,
)